In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy as sp
from sympy.vector import CoordSys3D

from matplotlib import animation, rc
from IPython.display import HTML, display
rc('animation', html='jshtml')
import seaborn as sns
C = CoordSys3D('')

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('poster', font_scale=0.8,  rc={"lines.linewidth": 5})

## Problema (Mola atrito)

<font size=3px>Uma mola de constante elástica $k = 20$ N/m está presa a um bloco de massa $m = 5$ kg. O bloco é
    colocado na posição $x_0 = -1$ m e solto a partir do repouso. A mola encontra-se relaxada quando a posição do bloco
    é $x=0$. Se a superfície possui um coeficiente de atrito cinético $\mu_k = 0.02$, calcule:<br>
    a) O trabalho realizado pela força de atrito entre a $x_0$ e a elongação máxima da mola ($x_{max}$). Escreva a resposta em função de $x_{max}$.<br>
    b) O trabalho realizado pela força elástica entre a $x_0$ e a elongação máxima da mola ($x_{max}$).  Escreva a resposta em função de $x_{max}$.<br>
    c) Determine o valor de $x_{max}$.<br>
    d) Determine para qual valor de $x$ a energia cinética do bloco será máxima.
</font>

In [2]:
#Definir variaveis/parametros:
m,k,x0,v,x,xm,muk,g = sp.symbols('m,k,x_0,v,x,x_{max},mu_k,g')
subsList = [(m,5.0),(x0,-1.),(k,20.),(muk,0.02),(g,9.8)]

In [3]:
#item a)
#Entre a posição inicial e xmax a força de atrito sempre é oposta ao eixo x e constante:
N = m*g
Fatv = -N*muk*C.i
#Então o trabalho da força será:
Wf = Fatv.dot((xm-x0)*C.i)
resp = display(Wf)
Wf.subs(subsList)

-g*m*mu_k*(-x_0 + x_{max})

-0.98*x_{max} - 0.98

In [4]:
#item b)
#A força elástica é dada por:
Fel = -k*x*C.i
#Então o trabalho da força será:
We = sp.integrate(Fel.dot(C.i),(x,x0,xm))
resp = display(We)
We.subs(subsList)

k*x_0**2/2 - k*x_{max}**2/2

10.0 - 10.0*x_{max}**2

In [5]:
#item c)
#xmax será dado pelo ponto onde a energia cinética é nula, ou seja deltaK = 0 (pois K0=0):
Wtotal = We + Wf
dK = 0
xmsol = sp.solve(sp.Eq(Wtotal,dK),xm)[1]
resp = display(xmsol)
xmsol.subs(subsList)

-2*g*m*mu_k/k - x_0

0.902000000000000

In [6]:
#item d)
#A energia cinética quando o bloco está na posição x é:
K = Wtotal.subs(xm,x)
resp = display(K)
xsol = sp.solve(sp.Eq(sp.diff(K,x),0),x)[0]
resp = display(xsol)
xsol.subs(subsList)

-g*m*mu_k*(x - x_0) - k*x**2/2 + k*x_0**2/2

-g*m*mu_k/k

-0.0490000000000000

## Problema (Pêndulo)

<font size=3px>Um pêndulo feito de uma esfera de massa $m$ presa a um fio de massa 
    desprezível e comprimento $l$, é solto do ponto indicado na figura. <br>
    a) Escreva o diagrama de forças que atuam no pêndulo quando este se encontra a um ângulo $\theta$.<br>
    b) Calcule o trabalho realizado pela tração durante o ponto inicial e o de menor altura do pêndulo.<br>
    c) Calcule o trabalho realizado pela força peso durante o ponto inicial e o de menor altura do pêndulo.<br>
    d) Qual o vetor velocidade do pêndulo quando ele se encontra em seu ponto de menor altura?
</font>
<img src="pendulo.png" alt="Drawing" style="width: 250px;"/>

In [35]:
#Definir variaveis/parametros:
m,l,tt,v,T,g,dtt,tt0 = sp.symbols('m,l,theta,v,T,g,dθ,theta_0')

In [32]:
#item a)
#Adotando um sistema onde y aponta para cima (vertical):
Tv = T*sp.cos(tt)*C.j - T*sp.sin(tt)*C.i
Pv = -m*g*C.j
resp = display(Tv)
resp = display(Pv)

(-T*sin(theta))*.i + (T*cos(theta))*.j

(-g*m)*.j

In [39]:
#item b)
#O vetor deslocamento é dado por:
drv = -l*dtt*(-sp.cos(tt)*C.i+-sp.sin(tt)*C.j)
#O produto escalar da tensão com dr é:
Tv.dot(drv)
#Logo WT = 0

0

In [40]:
#item c)
#O produto escalar do peso com dr é:
resp = display(Pv.dot(drv))
#Então o trabalho do peso é:
WP = sp.integrate(Pv.dot(drv)/dtt,(tt,tt0,0))
resp = display(sp.simplify(WP))

-dθ*g*l*m*sin(theta)

g*l*m*(1 - cos(theta_0))

In [42]:
#item d)
#No ponto de menor altura:
K = WP
vsol = sp.solve(sp.Eq(m*v**2/2,K),v)[0]
#E como a velocidade aponta contra o eixo x:
v = vsol*(-C.i)
v

(-sqrt(2)*sqrt(g*l*(1 - cos(theta_0))))*.i

## Problema (Mola - plano inclinado)

<font size=3px>Um bloco de massa $m=0,9$ kg parte com velocidade inicial $v_0$ (paralela ao plano)
    e desliza sobre um plano inclidano sem atrito que faz um ângulo de $\theta = 30^{\circ}$ com a horizontal,
    como mostrado na figura. Após deslizar uma distância $d= 70$ cm o bloco encontra uma mola
    não deformada e desliza mais 30 cm até que sua velocidade instantânea seja zero. A constante
    elástica da mola é $k = 190$ N/m. Adote $g = 10$ m/s$^2$.<br>
    a) Qual a velocidade do bloco ao encontrar a mola? Escreva a sua resposta em termos de $v_0$.<br>
    b) Qual a velocidade inicial do bloco ($v_0$)?
</font>
<img src="molaInclinado.png" alt="Drawing" style="width: 250px;"/>

## Problema (Blocos)

<font size=3px>O sistema representado na figura está em equilíbrio e θ = $60^\circ$.<br>
Considere agora que o fio 2 é cortado. Após algum tempo, cada um dos blocos oscilará como
um pêndulo. Se o comprimento do fio 1 é 10 cm, qual será a velocidade máxima atingida pelo
bloco de 100 kg durante seu movimento de oscilação? Despreze forças de atrito.<br>
</font>
<img src="blocosTensoes.jpg" alt="Drawing" style="width: 250px;"/>